In [32]:
import random
import math
import numpy as np

In [33]:
# XOR Data
x_seeds = np.array([(0,0), (1,0), (0,1), (1,1)], dtype = np.float64)
y_seeds = np.array([0,1,1,0])

In [34]:
N = 1000
idxs = np.random.randint(0,4,N)

In [35]:
X = x_seeds[idxs]
Y = y_seeds[idxs]

In [36]:
X += np.random.normal(scale = 0.25, size = X.shape)

In [37]:
class shallow_neural_network():
    def __init__(self, num_input_features, num_hiddens):
        self.num_input_features = num_input_features
        self.num_hiddens = num_hiddens

        self.W1 = np.random.normal(size=(num_hiddens, num_input_features))
        self.b1 = np.random.normal(size=num_hiddens)
        self.W2 = np.random.normal(size=num_hiddens)
        self.b2 = np.random.normal(size=1)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def predict(self, x):
        z1 = np.matmul(self.W1, x) + self.b1
        a1 = np.tanh(z1)
        z2 = np.matmul(self.W2, a1) + self.b2
        a2 = self.sigmoid(z2)
        return a2, (z1, a1, z2, a2)


In [38]:
model = shallow_neural_network(2,3)

In [39]:
# 기존 train 모델 (Vector 계산 없음)
def train1(X, Y, model, lr=0.1):
  """
  신경망 모델을 훈련하는 함수

  Args:
    X: 입력 데이터 (numpy 배열)
    Y: 레이블 데이터 (numpy 배열)
    model: 훈련할 신경망 모델 객체
    lr: 학습률 (기본값: 0.1)

  Returns:
    훈련 후의 손실 값
  """

  # 모델의 가중치 및 편향과 동일한 모양의 0으로 채워진 배열 초기화
  dW1 = np.zeros_like(model.W1)
  db1 = np.zeros_like(model.b1)
  dW2 = np.zeros_like(model.W2)
  db2 = np.zeros_like(model.b2)

  m = len(X)  # 데이터 샘플 수
  cost = 0.0  # 손실 초기화

  # 각 데이터 샘플에 대해 반복
  for x, y in zip(X, Y):
    # 모델 예측
    a2, (z1, a1, z2, _) = model.predict(x)

    # 손실 계산 (이진 교차 엔트로피)
    if y == 1:
      cost -= np.log(a2)
    else:
      cost -= np.log(1 - a2)

    # 출력층 오차 계산
    diff = a2 - y

    # 출력층 가중치 및 편향 업데이트
    db2 += diff

    for i in range(model.num_hiddens):
      dW2[i] += a1[i] * diff

    # 은닉층 오차 계산 및 가중치 및 편향 업데이트
    for i in range(model.num_hiddens):
      db1[i] += (1 - a1[i]**2) * model.W2[i] * diff
    for i in range(model.num_hiddens):
      for j in range(model.num_input_features):
        dW1[i, j] += x[j] * (1 - a1[i]**2) * model.W2[i] * diff

  # 평균 손실 계산
  cost /= m

  # 가중치 및 편향 업데이트
  model.W1 -= lr * dW1 / m
  model.b1 -= lr * db1 / m
  model.W2 -= lr * dW2 / m
  model.b2 -= lr * db2 / m

  return cost

In [40]:
# 변경된 train 모델 (Vector 계산 있음)
def train2(X, Y, model, lr=0.1):
    """
    신경망 모델을 훈련하는 함수

    Args:
        X: 입력 데이터 (numpy 배열)
        Y: 레이블 데이터 (numpy 배열)
        model: 훈련할 신경망 모델 객체
        lr: 학습률 (기본값: 0.1)

    Returns:
        훈련 후의 손실 값
    """

    # 모델의 가중치 및 편향과 동일한 모양의 0으로 채워진 배열 초기화
    dW1 = np.zeros_like(model.W1)
    db1 = np.zeros_like(model.b1)
    dW2 = np.zeros_like(model.W2)
    db2 = np.zeros_like(model.b2)

    m = len(X)  # 데이터 샘플 수
    cost = 0.0  # 손실 초기화

    # 순전파 및 역전파
    for x, y in zip(X, Y):
        # 모델 예측
        a2, (z1, a1, z2, _) = model.predict(x)

        # 손실 계산 (이진 교차 엔트로피)
        cost -= np.log(a2) if y == 1 else np.log(1 - a2)

        # 출력층 오차 계산
        diff = a2 - y

        # 출력층 가중치 및 편향 업데이트 (벡터 연산 적용)
        db2 += diff
        dW2 += np.outer(a1, diff).reshape(dW2.shape)  # a1 * diff의 벡터 연산

        # 은닉층 오차 계산 및 가중치 및 편향 업데이트 (벡터 연산 적용)
        delta1 = (1 - a1**2) * model.W2 * diff  # 은닉층 오차 계산
        db1 += delta1
        dW1 += np.outer(delta1, x)  # x * delta1의 벡터 연산

    # 평균 손실 계산
    cost /= m

    # 가중치 및 편향 업데이트
    model.W1 -= lr * dW1 / m
    model.b1 -= lr * db1 / m
    model.W2 -= lr * dW2 / m
    model.b2 -= lr * db2 / m

    return cost


In [41]:
for epoch in range(100):
  cost = train2(X,Y, model, 1.0)
  if epoch % 10 == 0:
    print(epoch, cost)

0 [1.18064132]
10 [0.65943083]
20 [0.62982477]
30 [0.59894456]
40 [0.56832475]
50 [0.54099461]
60 [0.5178652]
70 [0.49821815]
80 [0.48013423]
90 [0.45871102]


In [42]:
model.predict((0,0))[0].item()

0.4578090594042799

In [43]:
model.predict((0,1))[0].item()

0.5438960869513847

In [44]:
model.predict((1,0))[0].item()

0.92166610869742

In [45]:
model.predict((1,1))[0].item()

0.0926880560374332